# SQL 3  -- Aggregation and Grouping
This notebook provides code from the SQL2 lecture


## Initialization

Run the next cell to set up PostgreSQL

In [ ]:
# install
!pip install psycopg2-binary
!apt install postgresql postgresql-contrib &>log

## Connect to a database 


In [ ]:
# Set your database configuation information
# set this value to your database's username
dbuser = "netId"
# set this value to your database's name
dbName = "netIddb"
# set this value to your database's port
port = 5432
# set this value to your database's endpoint
endpoint = "postgres.clear.rice.edu"


In [ ]:
# build the connection string
def make_conn_str(dbuser, password, endpoint, port, dbName):

    return f"postgresql+psycopg2://{dbuser}:{password}@{endpoint}:{port}/{dbName}"

        
import getpass
password = getpass.getpass()

In [ ]:
# set connection
%load_ext sql
conn_str = make_conn_str(dbuser, password,endpoint, port, dbName)
# Limit queries to 100 results. Increase this value if needed, but recognize that your JN will increase in size as well. 
%config SqlMagic.displaylimit=100
%sql $conn_str

### Create the tables


In [ ]:
%%sql
DROP TABLE IF EXISTS Frequents;
CREATE TABLE Frequents
(
    drinker VARCHAR(50) NOT NULL,
    cafe VARCHAR(50) NOT NULL,
    CONSTRAINT frequents_pkey PRIMARY KEY (drinker, cafe)
);

DROP TABLE IF EXISTS Likes;
CREATE TABLE Likes
(
    drinker VARCHAR(50) NOT NULL,
    coffee VARCHAR(50) NOT NULL,
    PRIMARY KEY (drinker, coffee)
);

DROP TABLE IF EXISTS Serves CASCADE;
CREATE TABLE Serves
(
    cafe VARCHAR(50) NOT NULL,
    coffee VARCHAR(50)  NOT NULL,
    PRIMARY KEY (cafe, coffee)
);

DROP TABLE IF EXISTS Rates CASCADE;
CREATE TABLE Rates
(
    drinker VARCHAR(50) NOT NULL,
    coffee VARCHAR(50) NOT NULL,
    score INTEGER NOT NULL
);


Load some data

In [ ]:
%%sql
DELETE FROM Frequents;
DELETE FROM Likes;
DELETE FROM SERVES;
DELETE FROM Rates;

INSERT INTO Frequents(drinker, cafe) VALUES 
('Chris', 'A Cafe'),
('Chris', 'Double Trouble'),
('Risa', 'Brew Joint'),
('Risa', 'Java Lava'),
('Risa', 'Double Trouble');

INSERT INTO Likes(drinker, coffee) VALUES 
('Chris', 'Drip'),
('Chris', 'Espresso'),
('Risa', 'Cold Brew'),
('Risa', 'Drip'),
('Risa', 'Espresso'),
('Carlos', 'Cappuccino'),
('Ying', 'Cold Brew'),
('Ying', 'Drip'),
('Ying', 'Espresso'),
('Ying', 'Cappuccino');

INSERT INTO Serves(cafe, coffee) VALUES 
('A Cafe', 'Espresso'),
('A Cafe', 'Cold Brew'),
('Brew Joint', 'Espresso'),
('Double Trouble', 'Espresso'),
('Double Trouble', 'Cold Brew');


INSERT INTO Rates(drinker, coffee, score) VALUES
('Chris', 'Cold Brew', 1),
('Chris', 'Turkish Coffee', 5),
('Jorge', 'Cold Brew', 1),
('Jorge', 'Chai Latte', 3),
('Risa', 'Cold Brew', 4),
('Risa', 'Cold Brew', 5),
('Risa', 'Espresso', 2);




## Aggregation

```RATES (DRINKER, COFFEE, SCORE)```


What is the average coffee rating given by Risa?

In [ ]:
%%sql
SELECT  AVG(r.score)
FROM Rates r
WHERE r.drinker = 'Risa'


 Note: Rates does not have a primary key
 
 What are the repercussions?

-- your thoughts here

How many coffees has Risa rated?

In [ ]:
%%sql
SELECT COUNT (*)
FROM Rates r
WHERE r.drinker = 'Risa'

What does this return?

-- your thoughts here

This gives us the actual number rated, not the number of different coffees.

How could you change this query to return the number of different coffees rates?

In [ ]:
%%sql
SELECT COUNT (*)
FROM Rates r
WHERE r.drinker = 'Risa'

Given the following contents of the RATES relation:

In [ ]:
%%sql
SELECT r.*
FROM Rates r

What is the average rating for **each** coffee?

Enter your prediction, then run the query.

I predict:


In [ ]:
%%sql
SELECT r.coffee, AVG (r.score)
FROM Rates r
GROUP BY r.coffee

* This first groups the relation into subgroups
* Every tuple in the subgroup has the same value for r.coffee
* Then the aggregate runs over each subgroup independently


What is each drinker's average coffee rating?

In [ ]:
%%sql
SELECT r.drinker, AVG(r.score) AS avgScore
FROM Rates r
GROUP BY r.drinker

### GROUP BY

1. Sorts the data into groups, by the specified attributes

1. Computes the aggregate function on each group

### HAVING

The ```HAVING``` clause applies a selection to each group


### GROUP BY and HAVING example

 What is the highest rated type of coffee, on average, considering only coffees that have at least 3 ratings?
  

In [ ]:
%%sql
DROP VIEW IF EXISTS COFFEE_AVG_RATING;
CREATE VIEW COFFEE_AVG_RATING AS
   SELECT r.coffee, AVG (r.score) AS avgScore
   FROM Rates r
   GROUP BY r.coffee;
    
SELECT a.coffee
FROM COFFEE_AVG_RATING a
WHERE a.avgScore = (SELECT MAX(a.avgScore)
                      FROM COFFEE_AVG_RATING a)


Now check for at least 3 ratings.

In [ ]:
%%sql
DROP VIEW IF EXISTS COFFEE_AVG_RATING;
CREATE VIEW COFFEE_AVG_RATING AS
   SELECT r.coffee, AVG(r.score) AS avgScore
   FROM Rates r
   GROUP BY COFFEE
   HAVING COUNT(*) >= 3

and rerun the query:

In [ ]:
%%sql
SELECT a.coffee
FROM COFFEE_AVG_RATING a
WHERE a.avgScore = (SELECT MAX(a.avgScore)
                      FROM COFFEE_AVG_RATING a)


### Revisit subquery in FROM clause

These subqueries can be complex. 

 What is the highest rated coffee, on average?

In [ ]:
%%sql
SELECT a.coffee
FROM (SELECT r.coffee, AVG (r.score) AS AVG_RATING
      FROM Rates r
      GROUP BY r.coffee) a
WHERE a.AVG_RATING = (SELECT MAX(a.AVG_RATING)
                      FROM (SELECT r.coffee, AVG (r.score) 
                               AS AVG_RATING
                            FROM Rates r
                            GROUP BY r.coffee) a)


Let's rewrite it using a VIEW:

In [ ]:
%%sql
DROP VIEW IF EXISTS COFFEE_AVG_RATING;
CREATE VIEW COFFEE_AVG_RATING AS
SELECT r.coffee, AVG (r.score) AS avgScore
FROM Rates r
GROUP BY r.coffee;

SELECT a.coffee
FROM COFFEE_AVG_RATING a
WHERE a.avgScore = (SELECT MAX(a.avgScore)
                      FROM COFFEE_AVG_RATING a);
